In [1]:
import asyncio
from data_project.gsheets import DateSheet
from datetime import date, timedelta, datetime
import pandas as pd


target_date = (date.today() - timedelta(days=1))

async def create_sheet():
    return DateSheet('[RP_DMM] Revenue Report', date=target_date.isoformat())

task_sheet = asyncio.create_task(create_sheet())
await task_sheet
mysheet = task_sheet.result()

/Users/serenayu/Documents/data-project/data_project/gsheets.py:81: UserWarning: No columns named '日期' or 'date'.
  warnings.warn("No columns named '日期' or 'date'.")


In [2]:
# Change your sheet
mysheet.change_sheet("data", headers_row=2)
sheet = mysheet.worksheet

data = sheet.get_all_values()

# Assuming headers are in the second row (index 1)
headers = data[1]
data = data[2:]

# Create a DataFrame
df = pd.DataFrame(data, columns=headers)

# Filter rows where "Fan Leoi 1" column has the value "Bou Shek"
filtered_df = df[df['分類1'] == '不夜之城']

# Display the filtered DataFrame
print(filtered_df)

       Server         UID      Create Time Package ID      Package Name  \
11       1001  1001004732  18/4/2024 18:11       1166         コロネ石お得パック   
37       1011  1011006846   2/6/2024 19:13       1166         コロネ石お得パック   
56       1007  1007003598  26/4/2024 23:58       1171        星の羅針盤お得パック   
57       1007  1007003598  26/4/2024 23:58       1171        星の羅針盤お得パック   
59       1005  1005001574   22/4/2024 7:00       1167  SSR獣神の欠片 - セイレーン   
...       ...         ...              ...        ...               ...   
116272   1002  1002003747  18/4/2024 21:57       1456         コロネ石お得パック   
116287   1010  1010000164   8/5/2024 21:13       1456         コロネ石お得パック   
116307   1013  1013006958  26/7/2024 19:41       1456         コロネ石お得パック   
116332   1013  1013002018    9/7/2024 1:08       1474        星の羅針盤お得パック   
116374   1013  1013004604  17/7/2024 10:29       1474        星の羅針盤お得パック   

         Package Type price    Purchase Time   分類1   分類2         USD  \
11      TreasureHouse   500

In [ ]:
# select the target sheet
mysheet = DateSheet('nightless_city_test', date=target_date.isoformat())
mysheet.change_sheet('工作表1')
sheet = mysheet.worksheet
data = filtered_df.copy()
data = data.iloc[: 10]  # only update the first 10 rows

# define the relation of columns
column_names = {
    "userId": "userId",
    "serverPrefix": "serverPrefix",
    "Create Time": "CT",
}

# rename the col of data
data.columns = [column_names.get(c, c).lower() for c in data.columns]

# sort & filter columns
for header in mysheet.headers:
    if header not in data.columns:
        data.loc[:, header] = None
data = data[mysheet.headers]

# sort rows
# data['ct'] = data['ct'].str.replace(" ", "T").apply(datetime.fromisoformat)  # convert str into datetime
data.sort_values("ct", ascending=True, inplace=True)

# compute the cell to paste
cell = f"A{len(sheet.col_values(1)) + 1}"

# update the target sheet
sheet.update(cell, data.values.tolist())

In [6]:
filtered_df.merge?

Signature:
filtered_df.merge(
    right: 'DataFrame | Series',
    how: 'MergeHow' = 'inner',
    on: 'IndexLabel | AnyArrayLike | None' = None,
    left_on: 'IndexLabel | AnyArrayLike | None' = None,
    right_on: 'IndexLabel | AnyArrayLike | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool | None' = None,
    indicator: 'str | bool' = False,
    validate: 'MergeValidate | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

.. warning::

    If bot

In [8]:
df1 = pd.DataFrame([
    ['a', 'cn'],
    ['b', 'eu'],
], columns=['uid', 'country'])

df2 = pd.DataFrame([
    ['a', 0.99],
    ['a', 8],
    ['a', 12],
    #['b', 4]
], columns=['uid', 'price'])

print(df1)
print()
print(df2)

  uid country
0   a      cn
1   b      eu

  uid  price
0   a   0.99
1   a   8.00
2   a  12.00


In [11]:
df1.merge(df2, how='left', on='uid').fillna(0)

,uid,country,price
0,a,cn,0.99
1,a,cn,8.00
2,a,cn,12.00
3,b,eu,0.00


In [13]:
df3 = pd.concat([df1, df2])
df3

,uid,country,price
0,a,cn,NaN
1,b,eu,NaN
0,a,NaN,0.99
1,a,NaN,8.00
2,a,NaN,12.00
